In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# reading the data

In [ ]:
df=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='latin-1')

In [ ]:
df.columns=['label','message','notuseful1','notuseful2','notuseful3']

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['notuseful1','notuseful2','notuseful3'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label'].value_counts()/len(df['label'])*100

In [ ]:
plt.figure(figsize=(12,8))
df['label'].value_counts().plot(kind='bar',color='red')

In [ ]:
df['true_label']=df['label'].map({'ham':0,'spam':1})

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,8))
df['label'].value_counts().plot(kind='pie',shadow = True,explode=[0.2,0],autopct='%1.1f%%')
plt.ylabel("Spam vs Ham")
plt.legend(["Ham", "Spam"])
plt.show()

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
df['length']=df['message'].apply(len)

In [ ]:
df

In [ ]:
sns.set_style('darkgrid')
plt.style.use("fivethirtyeight")
plt.figure(figsize=(12,8))
df[df['label']=='ham']['length'].plot(kind='hist',bins=40,color='red',alpha=0.4)
df[df['label']=='spam']['length'].plot(kind='hist',bins=40,color='blue',alpha=0.8)

In [ ]:
df.groupby('label').describe()['length']

In [ ]:
df.sort_values('length',ascending=False)

In [ ]:
df.iloc[1084]['message']

In [ ]:
ham_message=df[df['label']=='ham']
spam_message=df[df['label']=='spam']

# creating word clouds

In [ ]:
from wordcloud import WordCloud, STOPWORDS

# ham messages

In [ ]:
def create_wordcloud(input_data):
    combined_text= ' '.join(input_data['message'].astype(str).tolist())
    stopwords = set(STOPWORDS) 
    show_wordcloud =WordCloud(stopwords=stopwords,background_color='white',max_words=3000,
                    colormap='viridis', width=800, height=600).generate(combined_text)
    plt.figure(figsize=(12,8), frameon=True)
    plt.imshow(show_wordcloud,interpolation='bilinear')
    plt.axis('off')
   
    plt.show()
    

In [ ]:
create_wordcloud(ham_message)


# Spam messages

In [ ]:
create_wordcloud(spam_message)

In [ ]:
import re
from string import punctuation
from nltk.corpus import stopwords
sw=stopwords.words('english')
print(sw)
len(sw)

In [ ]:
import nltk
lemmatizer=nltk.WordNetLemmatizer()

# Data Cleaning(remove punctuation and stopwords)Tokenization And Lemmatization

In [ ]:
def remove_punctuation_and_stopwords(text):
    #removing punctuation
    without_punctuation=[w for w in text if w not in punctuation]
    without_punctuation="".join(without_punctuation)
    
    # tokenizing
    tokens=re.split('\W+',without_punctuation)
    
    #removing stopwords
    without_punc_and_stopwords=[w for w in tokens if w.lower() not in sw]
    
    #lemmatization
    after_lemma=[lemmatizer.lemmatize(word) for word in without_punc_and_stopwords]
    return after_lemma
    


# Bag Of Words Approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(analyzer=remove_punctuation_and_stopwords)
X=vectorizer.fit_transform(df['message'])
print(X.shape)

In [ ]:
X

In [ ]:

y=df['true_label']

In [ ]:
from sklearn.model_selection import train_test_split

# Train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)
print('size of train set',X_train.shape,y_train.shape)
print('size of test set',X_test.shape,y_test.shape)

# Importing different types of algorithm for comparison

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Model Building

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [ ]:
clfs = {'SVC' : svc,'KN' : knc, 'NB': mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'BgC': bc, 'ETC': etc}

In [ ]:
def train_classifier(clf, X_train, y_train):    
    clf.fit(X_train, y_train)

In [ ]:
def predict_labels(clf, X_test):
    return (clf.predict(X_test))

# Training and Prediction

In [ ]:
pred_scores = []
for k,v in clfs.items():
    train_classifier(v, X_train, y_train)
    pred = predict_labels(v,X_test)
    pred_scores.append((k, accuracy_score(y_test,pred)))
pred_scores    

In [ ]:
max(pred_scores)


In [ ]:
df_pred = pd.DataFrame(pred_scores, columns=['model','Score'])
df_pred.set_index('model',inplace=True)

In [ ]:

df_pred.plot(kind='bar',ylim=(0.9,1.0), figsize=(11,6), align='center',colormap='autumn')